<a href="https://colab.research.google.com/github/tdiego74/TECO/blob/main/Categorizacion%20de%20Multas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from collections import defaultdict

# Descargar recursos necesarios de nltk
nltk.download('punkt')

# Ruta al archivo en Colab
file_path = '/content/motivos_multas.csv'

# Intentar cargar el archivo con codificación 'latin1' y manejar errores en las líneas
df = pd.read_csv(file_path, encoding='latin1', on_bad_lines='skip')

# Palabras clave y categorías predefinidas (ampliadas con los nuevos motivos)
palabras_clave_categorias = {
    "Semaforo": ["luces de semaforo", "luz de semaforo", "semaforo", "luz roja"],
    "Exceso de Velocidad": ["exceso", "velocidad"],
    "No Uso de Cinturón de Seguridad": ["cinturon", "seguridad", "correajes", "correajes de seguridad"],
    "No Respeto de Señales de Tránsito": ["señales", "girar", "carril", "bocacalle", "contramano", "sentido contrario"],
    "Estacionamiento Prohibido": ["Estac.", "estac.", "estacionar", "lugar prohibido", "reservado", "estacionamiento", "ticket", "doble fila", "triple fila", "ochava"],
    "Uso Indebido del Teléfono": ["telefonicos", "telefono", "celular", "mensaje de texto", "redactando", "enviando"],
    "Falta de Documentación": ["cedula vencida", "documentacion", "revisión técnica", "licencia", "documentación", "revision"],
    "Uso Incorrecto de Luces": ["luces", "freno", "dirección", "retroceso", "niebla", "emergencia", "baja", "alta", "sucias", "encendidas", "apagadas"],
    "Obstrucción de Tránsito": ["senda", "peatonal", "obstruir", "obstruccion", "circulación", "obstaculizar"],
    "Carga y Descarga Indebida": ["carga", "descarga", "fuera de hora", "hora/lugar prohibido"],
    "Otras Infracciones": ["maniobra peligrosa", "conducir peligroso", "no detenerse", "uso de luces"]
}

# Definir la prioridad de las categorías
prioridad_categorias = [
    "Semaforo"
    "Exceso de Velocidad",
    "No Uso de Cinturón de Seguridad",
    "No Respeto de Señales de Tránsito",
    "Estacionamiento Prohibido",
    "Uso Indebido del Teléfono",
    "Falta de Documentación",
    "Uso Incorrecto de Luces",
    "Obstrucción de Tránsito",
    "Carga y Descarga Indebida",
    "Otras Infracciones"
]

# Función para asignar categoría y subcategoría con prioridad
def asignar_categoria(motivo):
    motivo_tokens = word_tokenize(motivo.lower())
    categorias_encontradas = set()

    for categoria, palabras_clave in palabras_clave_categorias.items():
        if any(palabra in motivo_tokens for palabra in palabras_clave):
            categorias_encontradas.add(categoria)

    if categorias_encontradas:
        # Encontrar la categoría de mayor prioridad
        categoria_asignada = min(categorias_encontradas, key=lambda x: prioridad_categorias.index(x))
        subcategoria_asignada = next((palabra for palabra in palabras_clave_categorias[categoria_asignada] if palabra in motivo_tokens), "")
    else:
        categoria_asignada = "Otras"
        subcategoria_asignada = ""

    return pd.Series([categoria_asignada, subcategoria_asignada])

# Aplicar la función a la columna de motivos
df[['Categoría', 'Subcategoría']] = df['Motivo'].apply(asignar_categoria)

# Identificar motivos sin categoría asignada y sugerir nuevas categorías
nuevos_motivos = df[df['Categoría'] == "Otras"]['Motivo'].unique()
nuevas_categorias_sugeridas = defaultdict(list)

for motivo in nuevos_motivos:
    motivo_tokens = word_tokenize(motivo.lower())
    for token in motivo_tokens:
        nuevas_categorias_sugeridas[token].append(motivo)

# Guardar el resultado en un nuevo archivo CSV
df.to_csv("/content/motivos_multas_categorizados.csv", index=False, encoding='utf-8-sig')

# Guardar nuevas categorías sugeridas en un archivo CSV
nuevas_categorias_df = pd.DataFrame.from_dict(nuevas_categorias_sugeridas, orient='index').transpose()
nuevas_categorias_df.to_csv("/content/nuevas_categorias_sugeridas.csv", index=False, encoding='utf-8-sig')

print("Categorías asignadas y guardadas en 'motivos_multas_categorizados.csv'")
print("Nuevas categorías sugeridas guardadas en 'nuevas_categorias_sugeridas.csv'")